In [1]:
import dash
from dash import dcc, html
import pandas as pd
from dash.dependencies import Input, Output
import plotly.graph_objs as go

df= pd.read_csv('job_skills.csv')
df0 = pd.read_csv('linkedin_job_postings.csv')
df1 =pd.merge(df0, df, on = 'job_link', how = 'inner')

# Get all unique companies from the dataset
selected_company = df1['company'].dropna().unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H3('Choose Company'),
    dcc.Dropdown(
        id='company-dropdown',
        options=[{'label': company, 'value': company} for company in selected_company],
        value=selected_company[0],
        style={'width': '50%'}
    ),
    dcc.Graph(id='company-comparison-chart')
])

@app.callback(
    Output('company-comparison-chart', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_chart(selected_company):
    company_data = df1[df1['company'] == selected_company]
    job_counts = company_data['search_position'].value_counts()

    # Create a bar graph
    bar_data = go.Bar(x=job_counts.index, y=job_counts.values)

    # Layout for the bar graph
    layout = go.Layout(title=f'Search Position Distribution for {selected_company}',
                       xaxis={'title': 'Search Position', 'automargin': True},
                       yaxis={'title': 'Count'})

    return {'data': [bar_data], 'layout': layout}

if __name__ == "__main__":
    app.run_server(debug=True, port=5091)
